In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
url = "https://en.wikipedia.org/wiki/List_of_highest-grossing_films#Highest-grossing_films"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
Data = []
id=0
#get data from wiki
for element in soup.find_all(["tbody"]):#find tag tbody
  if element.find("th", class_ ="unsortable"):#find tag th
    for row_element in element.find_all("th", scope = "row"):
      box = row_element.find_next("td")#find tag td
      box_office = box.find(string=True, recursive=False).replace("\n","").replace("$","")#get box_office
      release_year = box.find_next("td").get_text(strip = True)#gwt release_year
      link = "https://en.wikipedia.org/" + row_element.find("a").get('href')#find a link on a film
      res = requests.get(link)#get data from page of film
      s = BeautifulSoup(res.text, "html.parser")
      for inf in s.find_all("tbody"):#find tag tbody
        if inf.find("th", class_ = 'infobox-above summary')!=None:
          title = inf.find("th", class_ = 'infobox-above summary').get_text(strip=True)#find tag th
        if inf.find("th", class_ = 'infobox-label')!=None:
          if inf.find("th", class_ = 'infobox-label').get_text(strip=True) == "Directed by":#get directors
            author = inf.find("td", class_ = 'infobox-data')
            director=[]
            if author.find_all(string = True):
              if author.find_all('ul'):
                for i in author.find_all("li"):
                  director.append(i.find(string=True))
              elif title.count("Captain Marvel"):#exceptions
                for i in author.find("a").find_all(string= True):
                  director.append(i)
              else:
                for i in author:
                  if i.find(string=True, recursive=False):
                    director.append(i.find(string=True))
            if title.count("Captain America: Civil War"):#exceptions
              director.remove("Anthony Russo")
            if director.count(None):
              director.remove(None)
        if title.count("Pirate"):#exceptions
            country_header = inf.find('th', string=lambda text: text and "Country" in text)
            country_html = country_header.find_next("td")
            country = [country_html.find(string=True, recursive=False)]
            break#get title and directors name

        country_header = inf.find('th', string=lambda text: text and "Country" in text)#get country
        if country_header:
          country_html = country_header.find_next("td")
          country = [country_html.find(string=True, recursive=False)]
        country_header = inf.find('th', string=lambda text: text and "Countries" in text)
        if country_header:
          if title.count("7"):#exceptions
            country_html = country_header.find_next("td")
            country = [country_html.find(string=True, recursive=False)[:-1]]
          else:
            list = country_header.find_next("ul")
            country = []
            for li in list.find_all("li"):
              country.append(li.find(string=True, recursive=False))#get country
      id+=1
      film = (id, title, release_year, director, box_office, country)
      Data.append(film)
print(Data)#create a array with data

[(1, 'Avatar', '2009', ['James Cameron'], '2,923,706,026', ['United Kingdom', 'United States']), (2, 'Avengers: Endgame', '2019', ['Anthony Russo', 'Joe Russo'], '2,797,501,328', ['United States']), (3, 'Avatar: The Way of Water', '2022', ['James Cameron'], '2,320,250,281', ['United States']), (4, 'Titanic', '1997', ['James Cameron'], '2,257,844,554', ['United States']), (5, 'Star Wars: The Force Awakens', '2015', ['J. J. Abrams'], '2,068,223,624', ['United States']), (6, 'Avengers: Infinity War', '2018', ['Anthony Russo', 'Joe Russo'], '2,048,359,754', ['United States']), (7, 'Ne Zha 2', '2025', ['Anthony Russo', 'Joe Russo'], '1,983,302,560', ['China']), (8, 'Spider-Man: No Way Home', '2021', ['Jon Watts'], '1,922,598,800', ['United States']), (9, 'Inside Out 2', '2024', ['Kelsey Mann'], '1,698,863,816', ['United States']), (10, 'Jurassic World', '2015', ['Colin Trevorrow'], '1,671,537,444', ['United States']), (11, 'The Lion King', '2019', ['Jon Favreau'], '1,656,943,394', ['United 

In [14]:
import json
film_list = []
for film in Data:
    film_dict = {
        "id": film[0],
        "title": film[1],
        "release_year": film[2],
        "director": film[3],
        "box_office": film[4],
        "country": film[5]
    }
    film_list.append(film_dict)#structure of json
with open('film.json', 'w', encoding='utf-8') as f:
    json.dump(film_list, f, ensure_ascii=False, indent=2)#create json from array

In [2]:
import sqlite3

conn = sqlite3.connect('films.db')
cursor = conn.cursor()#create db

cursor.execute('''CREATE TABLE IF NOT EXISTS movies (
                    id INTEGER PRIMARY KEY,
                    title TEXT NOT NULL,
                    release_year  INTEGER,
                    director TEXT,
                    box_office TEXT,
                    country TEXT)''')#create table

for movie in Data:
    cursor.execute('''INSERT INTO movies
                      VALUES (?, ?, ?, ?, ?, ?)''',
                   (movie[0],
                    movie[1],
                    movie[2],
                    ', '.join(movie[3]),
                    movie[4],
                    ', '.join(movie[5])))#save data to table

conn.commit()
conn.close()#create a file with database from array